# JWST Image Gallery Organizer
**Astronomy 1221 - Project 2**  
**Authors:** Jacob Parzych & Niko Jamison  

---

## Project Overview
This project creates a browsable catalog of James Webb Space Telescope (JWST) observations with metadata organization and image visualization. We extract metadata from astronomical FITS files and organize them into Pandas DataFrames for analysis.

### Astronomy Context
The James Webb Space Telescope (JWST), launched in 2021, is the most powerful space telescope ever built, observing in infrared wavelengths. FITS (Flexible Image Transport System) is the standard astronomical image format containing both image data and extensive metadata in "headers." The `_i2d.fits` files we'll work with are processed, calibrated 2D images ready for scientific analysis.

## Step 1: Install Required Packages
First, we need to install the necessary packages for downloading and working with JWST FITS files.

In [1]:
# Install required packages
!pip install astroquery astropy pandas matplotlib numpy tqdm

## Step 2: Import Libraries
Import all necessary Python libraries for data manipulation, FITS file handling, and visualization.

In [2]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os
import warnings
warnings.filterwarnings('ignore')

# Astronomical data handling
from astropy.io import fits

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Astropy installed and ready for FITS file handling")

All libraries imported successfully!
Pandas version: 2.3.3
Astropy installed and ready for FITS file handling


## Step 3: Data Acquisition - Downloading JWST FITS Files

We'll use two approaches to download JWST data:
1. **Programmatic Download** using `astroquery` (demonstrates automation and reproducibility)
2. **Manual Download Instructions** for understanding the MAST Archive interface

### Approach 1: Programmatic Download using astroquery

We'll search for JWST observations of several famous targets and download their calibrated `_i2d.fits` files. These targets were chosen because they're iconic JWST "First Images" released in 2022 and have excellent data quality.

In [3]:
# Create directory for storing FITS files
data_dir = './jwst_data'
os.makedirs(data_dir, exist_ok=True)

print(f"Data directory created: {data_dir}")
print(f"Current working directory: {os.getcwd()}")

Data directory created: ./jwst_data
Current working directory: /Users/jacobparzych/astron1221/Astron_1221_JWST_Image_Gallery


### Download Strategy: Direct URLs from MAST Archive

Since MAST API queries can be unreliable, we'll use direct URLs to download specific JWST Early Release Observation (ERO) images. This approach is more reliable and similar to what was demonstrated in Lecture 14.

We'll download `_i2d.fits` files, which are:
- **Level 3 calibrated data products**
- **2D images** that have been fully processed and calibrated
- **Science-ready** for analysis
- The standard format used by professional astronomers

In [6]:
import urllib.request
import gzip
import shutil
from tqdm import tqdm

# Define JWST images to download from MAST archive
# These are specific JWST Early Release Observations (ERO) from famous targets
jwst_files = {
    # Southern Ring Nebula (NGC 3132) - NIRCam images
    "jw02732001001_02101_00001_nrcb1_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02732001001_02101_00001_nrcb1_i2d.fits",
        "target": "NGC 3132 (Southern Ring Nebula)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02732001001_02101_00002_nrcb2_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02732001001_02101_00002_nrcb2_i2d.fits",
        "target": "NGC 3132 (Southern Ring Nebula)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02732001001_02101_00001_nrcb3_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02732001001_02101_00001_nrcb3_i2d.fits",
        "target": "NGC 3132 (Southern Ring Nebula)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    # Carina Nebula (NGC 3324) - NIRCam images  
    "jw02731001001_02101_00001_nrcb1_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02731001001_02101_00001_nrcb1_i2d.fits",
        "target": "NGC 3324 (Carina Nebula)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02731001001_02101_00001_nrcb2_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02731001001_02101_00001_nrcb2_i2d.fits",
        "target": "NGC 3324 (Carina Nebula)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    # Stephan's Quintet - NIRCam images
    "jw02733001001_02101_00001_nrcb1_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02733001001_02101_00001_nrcb1_i2d.fits",
        "target": "Stephan's Quintet",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02733001001_02101_00001_nrcb2_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02733001001_02101_00001_nrcb2_i2d.fits",
        "target": "Stephan's Quintet",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02733001001_02101_00001_nrcb3_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02733001001_02101_00001_nrcb3_i2d.fits",
        "target": "Stephan's Quintet",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    # SMACS 0723 (Deep Field) - NIRCam images
    "jw02736001001_02101_00001_nrcb1_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02736001001_02101_00001_nrcb1_i2d.fits",
        "target": "SMACS 0723 (Deep Field)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02736001001_02101_00001_nrcb2_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02736001001_02101_00001_nrcb2_i2d.fits",
        "target": "SMACS 0723 (Deep Field)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02736001001_02101_00001_nrcb3_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02736001001_02101_00001_nrcb3_i2d.fits",
        "target": "SMACS 0723 (Deep Field)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
    "jw02736001001_02101_00001_nrcb4_i2d.fits": {
        "url": "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/jw02736001001_02101_00001_nrcb4_i2d.fits",
        "target": "SMACS 0723 (Deep Field)",
        "instrument": "NIRCam",
        "filter": "F090W"
    },
}

print(f"Prepared {len(jwst_files)} JWST images for download from {len(set(info['target'] for info in jwst_files.values()))} different targets")
print("\nTarget breakdown:")
targets_count = {}
for info in jwst_files.values():
    target = info['target']
    targets_count[target] = targets_count.get(target, 0) + 1

for target, count in targets_count.items():
    print(f"  - {target}: {count} images")

Prepared 12 JWST images for download from 4 different targets

Target breakdown:
  - NGC 3132 (Southern Ring Nebula): 3 images
  - NGC 3324 (Carina Nebula): 2 images
  - Stephan's Quintet: 3 images
  - SMACS 0723 (Deep Field): 4 images


### Download FITS Files

Now we'll download each FITS file using `urllib.request`. This is the same approach used in Lecture 14 for downloading JWST data. The function checks if files already exist to avoid re-downloading.

In [7]:
def download_file(url, filename, data_dir):
    """
    Download a file from a URL to a local directory with progress bar.
    
    Parameters:
    -----------
    url : str
        URL to download from
    filename : str
        Local filename to save as
    data_dir : str
        Directory to save file in
    
    Returns:
    --------
    bool
        True if download succeeded, False otherwise
    """
    filepath = os.path.join(data_dir, filename)
    
    # Check if file already exists
    if os.path.exists(filepath):
        filesize_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"  ✓ {filename} already exists ({filesize_mb:.1f} MB)")
        return True
    
    try:
        # Download with progress bar (similar to Lecture 14)
        print(f"  Downloading {filename}...")
        
        class DownloadProgressBar(tqdm):
            def update_to(self, b=1, bsize=1, tsize=None):
                if tsize is not None:
                    self.total = tsize
                self.update(b * bsize - self.n)
        
        with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=filename) as t:
            urllib.request.urlretrieve(url, filepath, reporthook=t.update_to)
        
        filesize_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"  ✓ Downloaded {filename} ({filesize_mb:.1f} MB)")
        return True
        
    except Exception as e:
        print(f"  ✗ Error downloading {filename}: {e}")
        return False

# Download all FITS files
print("Starting JWST FITS file downloads...")
print("="*60)

downloaded_files = []
failed_files = []

for filename, info in jwst_files.items():
    print(f"\n{info['target']} - {info['instrument']} - {info['filter']}")
    success = download_file(info['url'], filename, data_dir)
    
    if success:
        downloaded_files.append(filename)
    else:
        failed_files.append(filename)

print("\n" + "="*60)
print("Download Summary:")
print(f"  ✓ Successfully downloaded/found: {len(downloaded_files)} files")
if failed_files:
    print(f"  ✗ Failed downloads: {len(failed_files)} files")
print("="*60)

Starting JWST FITS file downloads...

NGC 3132 (Southern Ring Nebula) - NIRCam - F090W
  ✓ jw02732001001_02101_00001_nrcb1_i2d.fits already exists (113.0 MB)

NGC 3132 (Southern Ring Nebula) - NIRCam - F090W
  ✓ jw02732001001_02101_00002_nrcb2_i2d.fits already exists (113.2 MB)

NGC 3132 (Southern Ring Nebula) - NIRCam - F090W
  ✓ jw02732001001_02101_00001_nrcb3_i2d.fits already exists (113.2 MB)

NGC 3324 (Carina Nebula) - NIRCam - F090W
  ✓ jw02731001001_02101_00001_nrcb1_i2d.fits already exists (113.0 MB)

NGC 3324 (Carina Nebula) - NIRCam - F090W
  ✓ jw02731001001_02101_00001_nrcb2_i2d.fits already exists (113.2 MB)

Stephan's Quintet - NIRCam - F090W
  ✓ jw02733001001_02101_00001_nrcb1_i2d.fits already exists (113.0 MB)

Stephan's Quintet - NIRCam - F090W
  ✓ jw02733001001_02101_00001_nrcb2_i2d.fits already exists (113.2 MB)

Stephan's Quintet - NIRCam - F090W
  ✓ jw02733001001_02101_00001_nrcb3_i2d.fits already exists (113.2 MB)

SMACS 0723 (Deep Field) - NIRCam - F090W
  ✓ jw027

jw02736001001_02101_00001_nrcb3_i2d.fits: 119MB [01:52, 1.06MB/s]                            
jw02736001001_02101_00001_nrcb3_i2d.fits: 119MB [01:52, 1.06MB/s]                           


  ✓ Downloaded jw02736001001_02101_00001_nrcb3_i2d.fits (113.2 MB)

SMACS 0723 (Deep Field) - NIRCam - F090W


jw02736001001_02101_00001_nrcb4_i2d.fits: 119MB [02:12, 901kB/s]                             

  ✓ Downloaded jw02736001001_02101_00001_nrcb4_i2d.fits (113.5 MB)

Download Summary:
  ✓ Successfully downloaded/found: 12 files


### Verify Downloaded Files

Let's verify that our FITS files were downloaded successfully by scanning the data directory and displaying file information.

In [8]:
# Scan for all FITS files in the data directory
all_fits_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.fits'):
            all_fits_files.append(os.path.join(root, file))

print(f"Found {len(all_fits_files)} FITS files in {data_dir}/")
print("\n" + "="*60)
print("File Inventory:")
print("="*60)

total_size_mb = 0
for i, filepath in enumerate(sorted(all_fits_files), 1):
    filename = os.path.basename(filepath)
    filesize_mb = os.path.getsize(filepath) / (1024 * 1024)
    total_size_mb += filesize_mb
    
    # Get target info if available
    if filename in jwst_files:
        target = jwst_files[filename]['target']
        print(f"{i:2d}. {filename:50s} | {filesize_mb:6.1f} MB | {target}")
    else:
        print(f"{i:2d}. {filename:50s} | {filesize_mb:6.1f} MB")

print("="*60)
print(f"Total data: {total_size_mb:.1f} MB ({total_size_mb/1024:.2f} GB)")
print("="*60)

Found 12 FITS files in ./jwst_data/

File Inventory:
 1. jw02731001001_02101_00001_nrcb1_i2d.fits           |  113.0 MB | NGC 3324 (Carina Nebula)
 2. jw02731001001_02101_00001_nrcb2_i2d.fits           |  113.2 MB | NGC 3324 (Carina Nebula)
 3. jw02732001001_02101_00001_nrcb1_i2d.fits           |  113.0 MB | NGC 3132 (Southern Ring Nebula)
 4. jw02732001001_02101_00001_nrcb3_i2d.fits           |  113.2 MB | NGC 3132 (Southern Ring Nebula)
 5. jw02732001001_02101_00002_nrcb2_i2d.fits           |  113.2 MB | NGC 3132 (Southern Ring Nebula)
 6. jw02733001001_02101_00001_nrcb1_i2d.fits           |  113.0 MB | Stephan's Quintet
 7. jw02733001001_02101_00001_nrcb2_i2d.fits           |  113.2 MB | Stephan's Quintet
 8. jw02733001001_02101_00001_nrcb3_i2d.fits           |  113.2 MB | Stephan's Quintet
 9. jw02736001001_02101_00001_nrcb1_i2d.fits           |  113.0 MB | SMACS 0723 (Deep Field)
10. jw02736001001_02101_00001_nrcb2_i2d.fits           |  113.2 MB | SMACS 0723 (Deep Field)
11. jw027

---

## ✅ Data Acquisition Complete!

We have successfully downloaded JWST `_i2d.fits` files from the MAST Archive. These are fully calibrated, science-ready images from the James Webb Space Telescope's Early Release Observations.

### What We Have:
- **Southern Ring Nebula (NGC 3132)**: Multiple NIRCam pointings showing this planetary nebula
- **Carina Nebula (NGC 3324)**: Star-forming region with iconic "Cosmic Cliffs"
- **Stephan's Quintet**: Interacting galaxy group

### Next Steps:
Now that we have the data, the next phases of the project will be:
1. **Extract metadata** from FITS headers (target name, coordinates, instrument, filter, exposure time, etc.)
2. **Organize into Pandas DataFrame** for easy manipulation and analysis
3. **Create catalog features** (filtering, sorting, summary statistics)
4. **Visualize the data** (plots, analysis dashboard)
5. **Display FITS images** (optional but recommended)

The data acquisition phase is complete. Time to start analyzing!

---

## 📊 Project Status

**Phase 1: Data Acquisition** ✅ **COMPLETE**

We have successfully:
- Set up the project environment and imported necessary libraries
- Created a data directory structure
- Downloaded 7 JWST `_i2d.fits` files using direct URLs from MAST
- Verified all files and confirmed successful downloads

**Total Data**: ~XX MB of JWST observations from 3 famous targets

### Ready for Next Phase!
The FITS files are now ready for:
1. Metadata extraction
2. DataFrame organization  
3. Catalog creation
4. Analysis and visualization

To continue the project, add new cells below to extract and analyze the FITS metadata.